<b> <h2> Capstone Project - Week 5 </h2></b>

<h5> This notebook contains the code for the entire Capstone project </h5>

<b><h4> Importing important modules </h4></b>

In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          84 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0     conda-forge

The following packages will be UPDATED:

    geopy:         1.11.0-py36_0 conda-forge --> 1.18.1-py_0 conda-forge


geopy-1.18.1         | 51 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executi

<b><h4> Downloading data </h4></b>

In [2]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


In [4]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

<h5> Filtering data </h5>

In [5]:
neighborhoods_data = newyork_data['features']

<h5> Transforming data into pandas dataframe </h5>

In [6]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Let's loop through the data and fill the dataframe one row at a time.

In [7]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Let's check resulting dataframe

In [8]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


<b><h4> Filtering out Manhattan neighborhood </h4></b>
Here, we are going to segment and cluster only the neighborhoods in Manhattan. 
So let's slice the original dataframe and create a new dataframe of Manhattan.

In [9]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [10]:
manhattan_data.shape

(40, 4)

<b><h2> Let's get Manhattan Neighborhood rental data now </h2> </b>
First let's import the necessary modules

In [11]:
import lxml
import html5lib
from bs4 import BeautifulSoup
import html5lib


In [12]:
df= pd.read_html('https://www.rentcafe.com/average-rent-market-trends/us/ny/manhattan/',flavor='bs4',header=0) 

In [13]:
rent_data=df[2]

In [14]:
rent_data.head()

,Average Rent,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Neighborhood,All rentals,Studio,1 Bed,2 Beds,3 Beds
1,Washington Heights,"$2,170","$1,678","$2,000","$2,600","$3,219"
2,East Harlem,"$2,528","$1,895","$2,362","$3,521","$5,755"
3,Harlem,"$2,783","$1,975","$2,660","$4,112","$6,765"
4,Ellis Island,"$3,328","$2,929","$3,384","$3,793","$1,424"


Let's consider the first row as Column headers 

In [15]:
new_header = rent_data.iloc[0] #grab the first row for the header
rent_data = rent_data[1:] #take the data less the header row
rent_data.columns = new_header #set the header row as the df header

In [16]:
rent_data.head()

,Neighborhood,All rentals,Studio,1 Bed,2 Beds,3 Beds
1,Washington Heights,"$2,170","$1,678","$2,000","$2,600","$3,219"
2,East Harlem,"$2,528","$1,895","$2,362","$3,521","$5,755"
3,Harlem,"$2,783","$1,975","$2,660","$4,112","$6,765"
4,Ellis Island,"$3,328","$2,929","$3,384","$3,793","$1,424"
5,Tudor City,"$3,389","$2,967","$3,556","$4,908","$5,450"


Now let's merge both the dataset across column 

In [17]:
merged_data = pd.merge(manhattan_data,rent_data, on="Neighborhood")

In [18]:
merged_data

,Borough,Neighborhood,Latitude,Longitude,All rentals,Studio,1 Bed,2 Beds,3 Beds
0,Manhattan,Chinatown,40.715618,-73.994279,"$4,864","$3,457","$4,689","$6,118","$12,517"
1,Manhattan,Washington Heights,40.851903,-73.936900,"$2,170","$1,678","$2,000","$2,600","$3,219"
2,Manhattan,East Harlem,40.792249,-73.944182,"$2,528","$1,895","$2,362","$3,521","$5,755"
3,Manhattan,Yorkville,40.775930,-73.947118,"$4,130","$2,616","$3,449","$5,520","$8,726"
4,Manhattan,Lenox Hill,40.768113,-73.958860,"$4,269","$2,475","$4,091","$5,917","$7,485"
5,Manhattan,Roosevelt Island,40.762160,-73.949168,"$3,430","$3,192","$3,285","$3,592","$5,375"
6,Manhattan,Upper West Side,40.787658,-73.977059,"$4,536","$2,750","$3,824","$5,722","$8,795"
7,Manhattan,Lincoln Square,40.773529,-73.985338,"$4,561","$2,896","$3,912","$6,498","$10,675"
8,Manhattan,Murray Hill,40.748303,-73.978332,"$3,883","$2,970","$3,641","$5,435","$6,758"
9,Manhattan,Chelsea,40.744035,-74.003116,"$4,181","$3,171","$4,205","$6,611","$10,992"


In [19]:
merged_data.shape

(25, 9)

Finding out geographical coordinates of Manhattan

In [20]:
address = 'Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


Let's create a map of Manhattan neighborhoods

In [21]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(merged_data['Latitude'], merged_data['Longitude'], merged_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

<b> Define Foursquare Credentials and Version<b>

**Hiding Forsquare credentials**

<b>Let's get the top 100 venues that are each neighborhood within a radius of 500 meters with their categories.</b>

Defining LIMIT & radius first

In [23]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

Creating function to extract the category of the venues

In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Creating function to get top 100 venues within radius 500 meters

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
manhattan_venues = getNearbyVenues(names=merged_data['Neighborhood'],
                                   latitudes=merged_data['Latitude'],
                                   longitudes=merged_data['Longitude']
                                  )

Chinatown
Washington Heights
East Harlem
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Little Italy
Manhattan Valley
Morningside Heights
Battery Park City
Financial District
Carnegie Hill
Civic Center
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Hudson Yards


Checking size of resulting dataframe

In [27]:
print(manhattan_venues.shape)
manhattan_venues.head()

(2085, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Chinatown,40.715618,-73.994279,SKY TING YOGA,40.716469,-73.995020,Yoga Studio
1,Chinatown,40.715618,-73.994279,Spicy Village 大福星 (Spicy Village),40.717010,-73.993530,Chinese Restaurant
2,Chinatown,40.715618,-73.994279,Mission Escape Games,40.716505,-73.994720,General Entertainment
3,Chinatown,40.715618,-73.994279,Bar Belly,40.715135,-73.991802,Cocktail Bar
4,Chinatown,40.715618,-73.994279,Kiki's,40.714476,-73.992036,Greek Restaurant


Now lets analyze each neighborhood

In [28]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soba Restaurant,Soup Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stables,Steakhouse,Street Art,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Tourist Information Center,Toy / Game Store,Track,Trail,Train,Tree,Turkish Restaurant,Udon Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,

In [29]:
manhattan_onehot.shape

(2085, 284)

Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [30]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soba Restaurant,Soup Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stables,Steakhouse,Street Art,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Tourist Information Center,Toy / Game Store,Track,Trail,Train,Tree,Turkish Restaurant,Udon Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,

Let's print each neighborhood along with the top 5 most common venues

In [31]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
                venue  freq
0         Coffee Shop  0.08
1                Park  0.07
2               Hotel  0.05
3           Wine Shop  0.03
4  Italian Restaurant  0.03


----Carnegie Hill----
                 venue  freq
0          Pizza Place  0.06
1       Cosmetics Shop  0.05
2          Coffee Shop  0.05
3                 Café  0.04
4  Japanese Restaurant  0.03


----Chelsea----
                venue  freq
0         Coffee Shop  0.07
1  Italian Restaurant  0.06
2      Ice Cream Shop  0.05
3              Bakery  0.04
4           Nightclub  0.04


----Chinatown----
                 venue  freq
0   Chinese Restaurant  0.09
1      Bubble Tea Shop  0.06
2         Cocktail Bar  0.04
3  American Restaurant  0.04
4   Dim Sum Restaurant  0.04


----Civic Center----
                  venue  freq
0  Gym / Fitness Center  0.06
1    Italian Restaurant  0.05
2                Bakery  0.05
3     French Restaurant  0.04
4          Cocktail Bar  0.03


----East Harlem----
   

Putting this into a pandas dataframe
First, let's write a function to sort the venues in descending order.


In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Creating the new dataframe and display the top 10 venues for each neighborhood

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Coffee Shop,Park,Hotel,Wine Shop,Italian Restaurant,Department Store,Plaza,Fountain,Shopping Mall,Sandwich Place
1,Carnegie Hill,Pizza Place,Coffee Shop,Cosmetics Shop,Café,Bar,French Restaurant,Spa,Bookstore,Japanese Restaurant,Yoga Studio
2,Chelsea,Coffee Shop,Italian Restaurant,Ice Cream Shop,Nightclub,American Restaurant,Bakery,Hotel,Seafood Restaurant,Art Gallery,Theater
3,Chinatown,Chinese Restaurant,Bubble Tea Shop,American Restaurant,Cocktail Bar,Vietnamese Restaurant,Dim Sum Restaurant,Salon / Barbershop,Hotpot Restaurant,Noodle House,Bakery
4,Civic Center,Gym / Fitness Center,Bakery,Italian Restaurant,French Restaurant,Yoga Studio,Sandwich Place,Park,Coffee Shop,Sporting Goods Shop,Cocktail Bar
5,East Harlem,Mexican Restaurant,Bakery,Latin American Restaurant,Deli / Bodega,Thai Restaurant,Convenience Store,Sandwich Place,Liquor Store,Street Art,Steakhouse
6,East Village,Bar,Ice Cream Shop,Wine Bar,Mexican Restaurant,Cocktail Bar,Speakeasy,Pizza Place,Ramen Restaurant,Chinese Restaurant,Café
7,Financial District,Coffee Shop,Hotel,Bar,Gym,Wine Shop,Steakhouse,Italian Restaurant,Pizza Place,Food Truck,Jewelry Store
8,Greenwich Village,Italian Restaurant,Sushi Restaurant,French Restaurant,Clothing Store,Indian Restaurant,Seafood Restaurant,Café,Chinese Restaurant,Bakery,Coffee Shop
9,Hudson Yards,Coffee Shop,Café,Italian Restaurant,Restaurant,Hotel,American Restaurant,Theater,Gym / Fitness Center,Park,Thai Restaurant


<b><h3> Cluster neighborhood </h3></b>
Running k-means to cluster the neighborhood into 5 clusters.

In [34]:
# set number of clusters
kclusters = 6

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([1, 0, 0, 5, 0, 3, 0, 1, 0, 1], dtype=int32)

Now creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [35]:
manhattan_merged = merged_data

# add clustering labels
manhattan_merged['Cluster Labels'] = kmeans.labels_

# merge manhattan_merged with manhattan neighborhoods venues to add latitude/longitude/rent for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,All rentals,Studio,1 Bed,2 Beds,3 Beds,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Chinatown,40.715618,-73.994279,"$4,864","$3,457","$4,689","$6,118","$12,517",1,Chinese Restaurant,Bubble Tea Shop,American Restaurant,Cocktail Bar,Vietnamese Restaurant,Dim Sum Restaurant,Salon / Barbershop,Hotpot Restaurant,Noodle House,Bakery
1,Manhattan,Washington Heights,40.851903,-73.936900,"$2,170","$1,678","$2,000","$2,600","$3,219",0,Café,Mobile Phone Shop,Bakery,Sandwich Place,Spanish Restaurant,Pizza Place,Mexican Restaurant,Gym,Tapas Restaurant,Grocery Store
2,Manhattan,East Harlem,40.792249,-73.944182,"$2,528","$1,895","$2,362","$3,521","$5,755",0,Mexican Restaurant,Bakery,Latin American Restaurant,Deli / Bodega,Thai Restaurant,Convenience Store,Sandwich Place,Liquor Store,Street Art,Steakhouse
3,Manhattan,Yorkville,40.775930,-73.947118,"$4,130","$2,616","$3,449","$5,520","$8,726",5,Bar,Gym,Italian Restaurant,Coffee Shop,Pizza Place,Sushi Restaurant,Deli / Bodega,Japanese Restaurant,Mexican Restaurant,Sandwich Place
4,Manhattan,Lenox Hill,40.768113,-73.958860,"$4,269","$2,475","$4,091","$5,917","$7,485",0,Coffee Shop,Italian Restaurant,Sushi Restaurant,Gym / Fitness Center,Pizza Place,Sporting Goods Shop,Gym,Burger Joint,Thai Restaurant,Cosmetics Shop


Let's visualize the resulting clusters

In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Let's examine clusters

<h3> Cluster 1 </h3>

In [126]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(4, manhattan_merged.shape[1]))]]


,Neighborhood,All rentals,Studio,1 Bed,2 Beds,3 Beds,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Washington Heights,"$2,170","$1,678","$2,000","$2,600","$3,219",0,Café,Mobile Phone Shop,Bakery,Sandwich Place,Spanish Restaurant,Pizza Place,Mexican Restaurant,Gym,Tapas Restaurant,Grocery Store
2,East Harlem,"$2,528","$1,895","$2,362","$3,521","$5,755",0,Mexican Restaurant,Bakery,Latin American Restaurant,Deli / Bodega,Thai Restaurant,Convenience Store,Sandwich Place,Liquor Store,Street Art,Steakhouse
4,Lenox Hill,"$4,269","$2,475","$4,091","$5,917","$7,485",0,Coffee Shop,Italian Restaurant,Sushi Restaurant,Gym / Fitness Center,Pizza Place,Sporting Goods Shop,Gym,Burger Joint,Thai Restaurant,Cosmetics Shop
6,Upper West Side,"$4,536","$2,750","$3,824","$5,722","$8,795",0,Italian Restaurant,Bar,Coffee Shop,Indian Restaurant,Wine Bar,Bakery,Vegetarian / Vegan Restaurant,Burger Joint,Cosmetics Shop,Gym / Fitness Center
8,Murray Hill,"$3,883","$2,970","$3,641","$5,435","$6,758",0,Hotel,Gym,Sandwich Place,Bar,Salon / Barbershop,Coffee Shop,French Restaurant,Italian Restaurant,Japanese Restaurant,Spa
10,Greenwich Village,"$4,197","$3,120","$4,216","$6,292","$11,221",0,Italian Restaurant,Sushi Restaurant,French Restaurant,Clothing Store,Indian Restaurant,Seafood Restaurant,Café,Chinese Restaurant,Bakery,Coffee Shop
11,East Village,"$4,174","$2,933","$4,131","$5,895","$5,687",0,Bar,Ice Cream Shop,Wine Bar,Mexican Restaurant,Cocktail Bar,Speakeasy,Pizza Place,Ramen Restaurant,Chinese Restaurant,Café
14,Manhattan Valley,"$4,553","$2,506","$3,347","$5,227","$7,932",0,Pizza Place,Coffee Shop,Yoga Studio,Thai Restaurant,Indian Restaurant,Italian Restaurant,French Restaurant,Mexican Restaurant,Deli / Bodega,Café
16,Battery Park City,"$5,363","$3,750","$4,237","$6,590","$10,386",0,Coffee Shop,Park,Hotel,Wine Shop,Italian Restaurant,Department Store,Plaza,Fountain,Shopping Mall,Sandwich Place
19,Civic Center,"$4,419","$3,026","$4,232","$6,305","$10,652",0,Gym / Fitness Center,Italian Restaurant,Bakery,French Restaurant,Yoga Studio,Sporting Goods Shop,Sandwich Place,Cocktail Bar,Coffee Shop,Park


<h3> Cluster 2 </h3>

In [127]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(4, manhattan_merged.shape[1]))]]

,Neighborhood,All rentals,Studio,1 Bed,2 Beds,3 Beds,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Chinatown,"$4,864","$3,457","$4,689","$6,118","$12,517",1,Chinese Restaurant,Bubble Tea Shop,American Restaurant,Cocktail Bar,Vietnamese Restaurant,Dim Sum Restaurant,Salon / Barbershop,Hotpot Restaurant,Noodle House,Bakery
7,Lincoln Square,"$4,561","$2,896","$3,912","$6,498","$10,675",1,Theater,Gym / Fitness Center,Concert Hall,Italian Restaurant,Plaza,French Restaurant,Café,Performing Arts Venue,Park,Indie Movie Theater
9,Chelsea,"$4,181","$3,171","$4,205","$6,611","$10,992",1,Coffee Shop,Italian Restaurant,Ice Cream Shop,Nightclub,American Restaurant,Bakery,Hotel,Seafood Restaurant,Art Gallery,Theater
15,Morningside Heights,"$4,326","$2,543","$3,408","$5,191","$7,978",1,Coffee Shop,Park,Food Truck,Bookstore,American Restaurant,Burger Joint,Sandwich Place,Tennis Court,Deli / Bodega,Pizza Place


<h3> Cluster 3 </h3>

In [128]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(4, manhattan_merged.shape[1]))]]

,Neighborhood,All rentals,Studio,1 Bed,2 Beds,3 Beds,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Carnegie Hill,"$4,130","$2,616","$3,449","$5,520","$8,726",2,Pizza Place,Coffee Shop,Cosmetics Shop,Café,Bar,French Restaurant,Spa,Bookstore,Japanese Restaurant,Yoga Studio


<h3> Cluster 4 </h3>

In [129]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(4, manhattan_merged.shape[1]))]]

,Neighborhood,All rentals,Studio,1 Bed,2 Beds,3 Beds,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Roosevelt Island,"$3,430","$3,192","$3,285","$3,592","$5,375",3,Park,Sandwich Place,Deli / Bodega,Baseball Field,Greek Restaurant,Residential Building (Apartment / Condo),Bus Station,Coffee Shop,Dry Cleaner,Bubble Tea Shop


<h3> Cluster 5 </h3>

In [130]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(4, manhattan_merged.shape[1]))]]

,Neighborhood,All rentals,Studio,1 Bed,2 Beds,3 Beds,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Financial District,"$4,018","$3,134","$4,045","$5,710","$9,575",4,Coffee Shop,Hotel,Bar,Gym,Wine Shop,Steakhouse,Italian Restaurant,Pizza Place,Food Truck,Jewelry Store
20,Sutton Place,"$4,000","$2,466","$3,702","$5,572","$7,697",4,Gym / Fitness Center,Italian Restaurant,Indian Restaurant,Furniture / Home Store,Juice Bar,American Restaurant,Bakery,Dessert Shop,Ice Cream Shop,Grocery Store
23,Stuyvesant Town,"$3,526","$2,632","$3,057","$3,883","$4,949",4,Bar,Playground,Park,Cocktail Bar,Gas Station,Basketball Court,Coffee Shop,German Restaurant,Harbor / Marina,Heliport
